From probability note

In [1]:
%%sh 
head -n 4 data/spam.csv #use bash command head to which lists first lines of the data

Couldn't find program: 'sh'


In [2]:
def load_sms():
    """
    A wrapper function to load the sms data
    """
    import csv
    lines = []
    hamspam = {'ham': 0, 'spam': 1}
    with open('data/spam.csv', mode='r', encoding='latin-1') as f:
        reader = csv.reader(f)
        # When using the csv reader, each time you use the function
        # next on it, it will spit out a list split at the ','
        header = next(reader)
        # We store this as ("txt",label), where we have used the function
        # hamspam to convert from "ham","spam" to 0 and 1.
        lines = [(line[1],hamspam[line[0]]) for line in reader]
        
    return lines

In [3]:
sms_data = load_sms() #get 0 for ham 1 for spam
sms_data[:2]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  0),
 ('Ok lar... Joking wif u oni...', 0)]

In [4]:
samplespace_sms_X = [0,1] #mycomment:possible values that x can take 0 for ham 1 for spam

probability of getting spam sms

In [5]:
# Number of times the spam occurs divided by the amount of sms data
N_spam_n = sum([sms[1] for sms in sms_data]) / len(sms_data)
N_spam_n

0.13406317300789664

In [6]:
relFreqs_sms_X = [1-N_spam_n, N_spam_n] #match each probability with its samples
sms_X_probmap = ProbyMap(sspace = samplespace_sms_X, probs=relFreqs_sms_X)# make our probability map
sms_X_probmap

NameError: name 'ProbyMap' is not defined

counting words probability

In [7]:
def free_in_txt(txt):
    if 'free' in txt: # txt = "freestyle", txt="free films" = 1, returns zero if we see txt="Free" 
        return 1
    return 0

In [8]:
sms_free_in_word = [free_in_txt(txt) for txt,label in sms_data] #count all the rows which has free in a sms

In [17]:
sum(sms_free_in_word)

122

In [18]:
#samplespace_sms_Y = samplespace_sms_X
# Number of times the free occured in data
N_free_n = sum(sms_free_in_word) / len(sms_free_in_word) #probability of getting free in the dataset
N_free_n

0.02189519023689878

conditional probability

𝑃(𝐴∣𝐵):=𝑃(𝐴∩𝐵)/𝑃(𝐵)

𝑃(𝑋=1∣𝑌=1):=𝑃(𝑋=1 and 𝑌=1)/𝑃(𝑌=1). here X is ham(0)/spam(1), Y is free(1)/no free(0)
 
What does this mean? What is the probability that our text is a spam given that we know it contains the word "free".

In [19]:
X = [label for txt,label in sms_data] #get output as 1,0-label output   
Y = sms_free_in_word
X_and_Y = [x*y for x,y in zip(X,Y)] #multiply to get 1*1 for X=1 and Y=1
N_X_and_Y = sum(X_and_Y)/len(X_and_Y) #probability of getting spam and free sms
N_X_and_Y

0.01094759511844939

In [20]:
# So lets just divide this with N_free_n from above and we get
N_X_and_Y/N_free_n #probability of getting spam sms if it has word 'free' in sms

0.5

Another way to do the same thing, is to take our original data and remove all texts that does not contain the word free and compute the relative frequency of spam in the remaining data

In [21]:
free_only_sms = [(txt, label) for txt,label in sms_data if free_in_txt(txt)]
sum([sms[1] for sms in free_only_sms])/len(free_only_sms)

0.5

Random variable note

Let X represents each SMS text (an entry in the list), and 
let  𝑌 represent whether text is spam or not i.e.  𝑌∈{0,1}

Thus  ℙ(𝑌=1) is the probability that we get a spam. The goal is to estimate:
ℙ(𝑌=1|"free" or "prize" is in 𝑋).
 
That is, the probability that the SMS is spam given that "free" or "prize" occurs in the SMS.

To do this we can create a new random variable  𝑍 which is  1 if "free" or "prize" appears in  𝑋.

In [22]:
interesting_words=set(['free','prize'])
TF10 = {True: 1, False: 0}
Z_obs = [TF10[not interesting_words.isdisjoint([word.lower() for word in line[0].split(' ')])] for line in sms_data] 
#my comment: list of values (either 0 or 1) based on whether certain interesting words are present in the lowercase words 
#of the text data in sms_data. 
#The values are determined by the TF10 dictionary, where True corresponds to 1 and False corresponds to 0.

In [23]:
Z_obs[:10]

[0, 0, 1, 0, 0, 0, 0, 0, 1, 1]

In [24]:
Y_obs = [y for x,y in sms_data]
Y_obs[:10]
#my comment: x text data,y label associated with text data.a list that contains the labels associated with the text data in the sms_data dataset. 
#If y represents the label (e.g., 0 or 1), Y_obs will be a list of these labels corresponding to each item in sms_data.

[0, 0, 1, 0, 0, 1, 0, 0, 1, 1]

In [25]:
import numpy as np
def F_X_12(x):
    TF10 = {True: 1, False: 0}
    return np.mean([TF10[(y <= x[0]) and (z <= x[1])] for y,z in zip (Y_obs,Z_obs)])

In [26]:
F_X_12([1,0])

0.9551328068916008

In [27]:
print("\tz <= 0 \t\tz <= 1")
for x1 in range(0,2):
    print("y <= %d \t" % x1,end='')
    for x2 in range(0,2):
        print("%.2f" % (F_X_12((x1,x2))),end='\t\t')
    print('\n') #joint distribution function for this problem

	z <= 0 		z <= 1
y <= 0 	0.86		0.87		

y <= 1 	0.96		1.00		



In [28]:
F_X_12((1,0))

0.9551328068916008

In [29]:
F_X_12((0,0)) == F_X_12((0,1))*F_X_12((1,0)) #check whether lhs=rhs

False

In [30]:
F_X_12((0,1))*F_X_12((1,0))

0.8270846721557743

In [31]:
 #Are they indepdentent? If so, then the JDF is just the product of the 
# DFs for Y and Z, but
0.865936826992103*0.955132806891601

0.827084672155774

Which is not 0.858, so they are not independent. So lets try to estimate the probability that  𝑌=1
  given that  𝑍=1 Lets again do that by filtering

In [32]:
np.mean([y for z,y in zip(Z_obs,Y_obs) if z == 1])

0.812

Compare that with the marginal probability of  𝑌=1, which is according to our JDF 1-0.866 = 0.134

In [33]:
np.mean([y for z,y in zip(Z_obs,Y_obs) if z == 1])

0.812

Recall that we computed the mean of Y_obs, which is the mean spam number. This is a Bernoulli random variable with unknown  𝑝
 , so we can use our methods above to compute a confidence interval

In [34]:
def epsilon_bernoulli(n,alpha):
    return np.sqrt(-1/(2*n)*np.log((alpha)/2))

In [35]:
epsilon = epsilon_bernoulli(len(Y_obs),0.05)
mean_Y_obs = np.mean(Y_obs)
print("[%.3f,%.3f]" % (mean_Y_obs-epsilon,mean_Y_obs+epsilon))

[0.116,0.152]


From this we get that we have provided a prediction as to what is the true probability of getting a spam email.

but this requires you to estimate both things on the right, and give a region for both and finally figure out an interval for the ratio. But, there is an easier way, which sometimes works better. Namely, to look at the random variable  𝑌∣(𝑍=1)
  which we do by filtering

In [37]:
Y_mid_Z1 = [y for z,y in zip(Z_obs,Y_obs) if z == 1]

epsilon = epsilon_bernoulli(len(Y_mid_Z1),0.05)
mean_Y_obs = np.mean(Y_mid_Z1)
print("[%.3f,%.3f]" % (mean_Y_obs-epsilon,mean_Y_obs+epsilon))

[0.726,0.898]


2022 paper

SMS spam filtering [8p]
In the following problem we will explore SMS spam texts. The dataset is the SMS Spam Collection Dataset and we have provided for you a way to load the data. If you run the appropriate cell below, the result will be in the spam_no_spam variable. The result is a list of tuples with the first position in the tuple being the SMS text and the second being a flag 0 = not spam and 1 = spam.

[3p] Let  𝑋
  be the random variable that represents each SMS text (an entry in the list), and let  𝑌
  represent whether text is spam or not i.e.  𝑌∈{0,1}
 . Thus  ℙ(𝑌=1)
  is the probability that we get a spam. The goal is to estimate:
ℙ(𝑌=1|"free" or "prize" is in 𝑋).
 
That is, the probability that the SMS is spam given that "free" or "prize" occurs in the SMS. Hint: it is good to remove the upper/lower case of words so that we can also find "Free" and "Prize"; this can be done with text.lower() if text a string.
[3p] Provide a "90%" interval of confidence around the true probability. I.e. use the Hoeffding inequality to obtain for your estimate  𝑃̂ 
  of the above quantity. Find  𝑙>0
  such that the following holds:
ℙ(𝑃̂ −𝑙≤𝔼[𝑃̂ ]≤𝑃̂ +𝑙)≥0.9.
 
[2p] Repeat the two exercises above for "free" appearing twice in the SMS.

In [38]:
spam_no_spam = load_sms()

In [46]:
import numpy as np

interesting_words=set(['free','prize'])
TF10 = {True: 1, False: 0}
Z_obs = [TF10[not interesting_words.isdisjoint([word.lower() for word in line[0].split(' ')])] for line in spam_no_spam]
Y_obs = [y for x,y in spam_no_spam]
X_obs = [X for x,y in spam_no_spam]
problem4_hatP=np.mean([y for z,y in zip(Z_obs,Y_obs) if z == 1])
# fill in the estimate for part 1 here (should be a number between 0 and 1)
problem4_hatP
print(X_obs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [40]:
def epsilon_bernoulli(n,alpha):
    return np.sqrt(-1/(2*n)*np.log((alpha)/2))

Y_mid_Z1 = [y for z,y in zip(Z_obs,Y_obs) if z == 1]

problem4_l = epsilon_bernoulli(len(Y_mid_Z1),0.1)

problem4_l
# fill in the calculated l from part 2 here
mean_Y_obs = np.mean(Y_mid_Z1)
print("[%.3f,%.3f]" % (mean_Y_obs-epsilon,mean_Y_obs+epsilon))

[0.726,0.898]


In [35]:
interesting_words=set(['free','free'])
TF10 = {True: 1, False: 0}
Z_obs2 = [TF10[not interesting_words.isdisjoint([word.lower() for word in line[0].split(' ')])] for line in spam_no_spam]
Y_obs2 = [y for x,y in spam_no_spam]
problem4_hatP2=np.mean([y for z,y in zip(Z_obs2,Y_obs2) if z == 1])

# fill in the estimate for hatP for the double free question in part 3 here (should be a number between 0 and 1)
problem4_hatP2

0.7626262626262627

In [38]:
# fill in the estimate for l for the double free question in part 3 here
Y_mid_Z2= [y for z,y in zip(Z_obs2,Y_obs2) if z == 1]
problem4_l2 = epsilon_bernoulli(len(Y_mid_Z2),0.1)
problem4_l2

0.08697689629305419

In [15]:
# if we have spam sms the probability of getting 'free' or 'prize' in that spam sms
def free_or_prize_in_txt(txt):
    if 'free' in txt.lower() or 'prize' in txt.lower():
        return 1
    return 0

free_or_prize_only_sms = [(txt, label) for txt,label in sms_data if free_or_prize_in_txt(txt)]
sum([sms[1] for sms in free_or_prize_only_sms])/len(free_or_prize_only_sms)

0.808695652173913

when we have two free words in sms how we get probability is as below

In [9]:
def free2_in_txt(txt):
    # Count the occurrences of the word 'free' in lowercase
    count_free = txt.lower().count('free')

    # Return 1 if the count is equal to or greater than 2
    if count_free == 2:
        return 1

    # Return 0 otherwise
    return 0

In [10]:
free2_only_sms = [(txt, label) for txt,label in sms_data if free2_in_txt(txt)]
sum([sms[1] for sms in free2_only_sms])/len(free2_only_sms)

0.9736842105263158

When two free and one prize appear in the dataset

In [18]:
def two_free_or_at_least_one_prize_in_txt(txt):
    # Count the occurrences of 'free' and 'prize' in lowercase
    count_free = txt.lower().count('free')
    count_prize = txt.lower().count('prize')

    # Return 1 if 'free' appears two or more times or 'prize' appears at least once
    if count_free == 2 or count_prize == 1:
        return 1

    # Return 0 otherwise
    return 0

In [19]:
free2_prize_sms = [(txt, label) for txt,label in sms_data if two_free_or_at_least_one_prize_in_txt(txt)]
sum([sms[1] for sms in free2_prize_sms])/len(free2_prize_sms)

0.9913793103448276

0.006640344580043073

0.9736842105263158